In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import seaborn as sns

import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot, plot

def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
def visulization(dataset, compares, regions, quarters, title_str):

    for region in regions:
        database = pd.DataFrame()
        for quarter in quarters:
            database = pd.concat([database, dataset[(dataset.REGION==region) & (dataset.YEAR_QUARTER==quarter)]]).reset_index(drop=True)

        data = []
        
        for k in compares:
            k = go.Scatter(
                x = list(range(len(database[str(k)]))),
                y = database[str(k)],
                mode = 'lines',
                name = str(k)
                )

            data.append(k)

        if 'DATE' in database.columns.values:
            width=1000
            height=600
            labels = database.DATE.unique()[::20]
            tickvals=[]
            for i in labels:
                tickvals.append(database[database.DATE == str(i)].index.tolist()[0]) 
                
            upper = go.Scatter(
                x = list(range(len(database['ACTUALS']))),
                y = database['ACTUALS'].values * 1.1,
                line = dict(color='black', width=4, dash='dash'),
                name = 'upper +10%'
                )

            lower = go.Scatter(
                x = list(range(len(database['ACTUALS']))),
                y = database['ACTUALS'].values * 0.9,
                line = dict(color='black', width=4, dash='dash'),
                name = 'lower -10%'
                )
            
            data.append(upper)
            data.append(lower)

        elif 'YEAR_QUARTER' in database.columns.values:
            width=1000
            height=400
            labels = database.YEAR_QUARTER.unique()
            tickvals=[]
            for i in labels:
                tickvals.append(database[database.YEAR_QUARTER == str(i)].index.tolist()[0])

        elif ('SnapDate' not in database.columns.values) and ('YEAR_QUARTER' not in database.columns.values):
            fig = go.Figure(data)
            iplot(fig)

        layout = go.Layout(
            title= str(region)  + ' - ' + title_str,

            autosize=False,width=width,height=height,

            xaxis=go.layout.XAxis(
                #title = str(compares),
                tickangle=75,
                ticktext=labels,
                tickvals=tickvals,
                showticklabels=True
            ),
            yaxis= dict(
                overlaying='y',
                side='left',
                showgrid=False,
                showticklabels=True
            )
        )

        fig = go.Figure(data, layout)

        iplot(fig)



In [10]:
df_final = pd.read_csv('./df_final.csv', sep = ';', decimal=',', index_col=0)

df_final_region = pd.read_csv('./df_final_region.csv', sep = ';', decimal=',', index_col=0)

mape = pd.read_csv('./mape.csv', sep = ';', decimal=',', index_col=0)

# Comparison and visualization

## Forecasting result: global VS regional

In [5]:
mape.columns

Index(['REGION', 'YEAR_QUARTER', 'MAPE_final_fc_region',
       'MAPE_mod1_lasso_region', 'MAPE_mod1_rf_region',
       'MAPE_mod2_lasso_region', 'MAPE_mod2_rf_region', 'MAPE_final_fc',
       'MAPE_mod1_lasso', 'MAPE_mod1_rf', 'MAPE_mod2_lasso', 'MAPE_mod2_rf'],
      dtype='object')

In [7]:
dataset = mape
compares = ['MAPE_mod1_lasso','MAPE_mod1_rf','MAPE_mod1_lasso_region','MAPE_mod1_rf_region']   
regions = ['AP','LA','MEE','GCN','NOAM','EMNORTH', 'EMSOUTH']
quarters = ['2018-Q2','2018-Q3','2018-Q4','2019-Q1','2019-Q2']
title_str = 'mape- model 1 forecasting'

visulization(dataset, compares, regions, quarters, title_str)

In [8]:
dataset = mape
compares = ['MAPE_final_fc_region','MAPE_final_fc']   #['test_y','LassoReg_Pred', 'RF_Pred']
regions = ['AP','LA','MEE','GCN','NOAM','EMNORTH', 'EMSOUTH']
quarters = ['2018-Q2','2018-Q3','2018-Q4','2019-Q1','2019-Q2']
title_str = 'mape - Final forecasting'

visulization(dataset, compares, regions, quarters, title_str)

In [154]:
df_final.columns

Index(['DATE', 'REGION', 'YEAR_QUARTER', 'DV2', 'lr_Pred', 'lasso_Pred',
       'rf_Pred', 'DV', 'lr_Pred_mod2', 'lasso_Pred_mod2', 'rf_Pred_mod2',
       'ESTIMATED_IN', 'ACTUALS', 'final_fc', 'MAPE_final_fc',
       'MAPE_mod1_lasso', 'MAPE_mod1_rf', 'MAPE_mod2_lasso', 'MAPE_mod2_rf'],
      dtype='object')

In [9]:
dataset = df_final
compares = ['final_fc']   #['test_y','LassoReg_Pred', 'RF_Pred']
regions = ['AP','LA','MEE','GCN','NOAM','EMNORTH', 'EMSOUTH']
quarters = ['2018-Q2','2018-Q3','2018-Q4','2019-Q1','2019-Q2']
title_str = 'Final forecasting global'

visulization(dataset, compares, regions, quarters, title_str)

In [159]:
df_final_region.columns

Index(['DATE', 'REGION', 'YEAR_QUARTER', 'DV2', 'regional_lr_Pred',
       'regional_lasso_Pred', 'regional_rf_Pred', 'DV',
       'regional_lr_Pred_mod2', 'regional_lasso_Pred_mod2',
       'regional_rf_Pred_mod2', 'ESTIMATED_IN', 'ACTUALS', 'final_fc_region',
       'MAPE_final_fc_region', 'MAPE_mod1_lasso_region', 'MAPE_mod1_rf_region',
       'MAPE_mod2_lasso_region', 'MAPE_mod2_rf_region'],
      dtype='object')

In [160]:
dataset = df_final_region
compares = ['final_fc_region']   #['test_y','LassoReg_Pred', 'RF_Pred']
regions = ['AP','LA','MEE','GCN','NOAM','EMNORTH', 'EMSOUTH']
quarters = ['2018-Q2','2018-Q3','2018-Q4','2019-Q1','2019-Q2']
title_str = 'Final forecasting regional'

visulization(dataset, compares, regions, quarters, title_str)

# check per quarter per region

In [15]:
dataset = df_final_region
compares = ['final_fc_region']   #['test_y','LassoReg_Pred', 'RF_Pred']
regions = ['AP','LA','MEE'] #,'GCN','NOAM','EMNORTH', 'EMSOUTH']
quarters = ['2018-Q2','2018-Q3'] #,'2018-Q4','2019-Q1','2019-Q2']
title_str = 'Final forecasting regional'

for quarter in quarters:
        quarter = [quarter]
        visulization(dataset, compares, regions, quarter, title_str)